In [1]:
!pip uninstall kafka -y

In [2]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.3/326.3 kB 2.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
# producer.py
# Ghi: pip install kafka-python
import json
import logging
import random
import time
from datetime import datetime
from kafka import KafkaProducer

# Cấu hình logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(name)s - %(message)s"
)
logger = logging.getLogger("kafka-producer")

KAFKA_BOOTSTRAP = "kafka1:9092"
TOPIC = "events.topic"

def create_producer(bootstrap_servers=KAFKA_BOOTSTRAP):
    producer = KafkaProducer(
        bootstrap_servers=bootstrap_servers,
        value_serializer=lambda v: json.dumps(v).encode("utf-8"),
        key_serializer=lambda k: k.encode("utf-8") if k is not None else None,
        linger_ms=50,
        acks='all'  # đảm bảo durability
    )
    logger.info("Kafka producer created (bootstrap=%s)", bootstrap_servers)
    return producer

def gen_event(i):
    # Tạo event mẫu - bạn thay schema cho phù hợp
    event = {
        "id": f"evt-{i}",
        "created_at": datetime.utcnow().isoformat() + "Z",
        "user_id": random.randint(1, 1000),
        "amount": round(random.random() * 1000, 2),
        "category": random.choice(["click", "purchase", "login"])
    }
    return event

def main():
    producer = create_producer()
    i = 0
    try:
        while True:
            event = gen_event(i)
            key = event["id"]
            # send và callback
            future = producer.send(TOPIC, key=key, value=event)
            try:
                record_metadata = future.get(timeout=10)
                logger.info("Sent event id=%s to %s partition=%s offset=%s",
                            key, record_metadata.topic, record_metadata.partition, record_metadata.offset)
            except Exception as e:
                logger.exception("Failed to send event id=%s: %s", key, e)

            i += 1
            time.sleep(0.5)  # gửi 2 event / giây (tùy chỉnh)
    except KeyboardInterrupt:
        logger.info("Stopping producer...")
    finally:
        producer.flush()
        producer.close()
        logger.info("Producer closed.")

if __name__ == "__main__":
    main()
